In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 8, 25)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-08-25
yesterday: 2023-08-24


### Tables in the process

In [3]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [4]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,JMART,158,2023-04-19,"1,200",19.60,1050,1252,2023-08-23,20.60,51.17,3.58,"24,665.25",126,"1,093.15",4.64%,13.43%,-50,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [5]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'JMART'

In [6]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1252



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,JMART,1252,158,2023-04-19,"1,200",19.60,48.69,3.41,Sold,"23,572.10",DTD,None


In [7]:
transaction, name, qty, sell_price, buy_price

('S', 'JMART', -1200, 20.6, 19.6)

### In case of buy, retreive buy id from latest buy record (id)

In [5]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,GVREIT,1280,169,2023-08-25,"6,000",7.35,91.29,6.39,Active,"44,197.68",HD,None


In [6]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'GVREIT', 6000, 7.35)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [8]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,period,grade
0,JMART,2023-01-20,"7,200",32.00,1,4,C3


In [9]:
#qty = -30000
new_cost = 34.5
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -1200, price = 34.5
WHERE name = 'JMART'



In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,period,grade
0,JMART,2023-01-20,"6,000",34.50,1,4,C3


### If shares become zero, must delete from buy table

In [48]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'ICHI' AND volbuy = 0


In [49]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.0000,0.0000,0.6600,"7,200",2023-08-24,2023-09-08,,0


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -1200
WHERE name = 'JMART'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.0000,0.0000,0.6600,"6,000",2023-08-24,2023-09-08,,0


### If shares become zero and want to delete from DIVIDEND table

In [50]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s'
"""
sqlDel = sqlDel % (name)
print(sqlDel)


DELETE FROM dividend
WHERE name = 'ICHI'



In [51]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [16]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'JMART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,111,JMART,0.00,0.00,I,0.00,20.60,0.00,0.00,32.00,"1,200",-4,4,"7,200",0.000000,0.000000,5pct,SET50


In [17]:
buy_target  = 18.9 # must calculate manually beforehand
sell_target = 23.5
cost = 34.5
status = 'B'
reason = "RD45pct"
sell_qty = 3000

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, cost = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, cost, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -1200,
qty = 3000, buy_target = 18.9, sell_target = 23.5, cost = 34.5, reason = 'RD45pct', status = 'B'
WHERE name = 'JMART'



In [18]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [19]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'JMART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,111,JMART,0.00,0.00,B,18.90,23.50,0.00,0.00,34.50,"3,000",-4,4,"6,000",0.000000,0.000000,RD45pct,SET50


### End of Update stocks table in SQLite port_lite database from sale transaction

### If shares become zero and no longer wants to keep STOCK record

In [24]:
sqlDel = """
DELETE FROM stocks
WHERE name = '%s' AND available_qty <= 0
"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM stocks
WHERE name = 'BANPU' AND available_qty <= 0



In [25]:
rp = conlite.execute(sqlDel)
rp.rowcount

1

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [7]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'GVREIT'



,name,date,shares,price,active,dividend,period,grade
0,GVREIT,2022-08-24,36000,8.7,1,0.792,2,B3


### In case there is the same stock name in port

In [8]:
#qty = 1200
new_cost = 8.50 # new average cost must be calculated manually beforehand
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 6000, price = 8.5
WHERE name = 'GVREIT'



In [9]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [10]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'GVREIT'



,name,date,shares,price,active,dividend,period,grade
0,GVREIT,2022-08-24,"42,000",8.50,1,0.7920,2,B3


### In case of totally new stock

In [8]:
rcd = ['AIMIRT', '2023-08-17', 10000, 11, 0, 0, 1, 0, 2,'A3']
rcd

['AIMIRT', '2023-08-17', 10000, 11, 0, 0, 1, 0, 2, 'A3']

In [9]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [10]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'AIMIRT'



,name,date,shares,price,active,dividend,period,grade
0,AIMIRT,2023-08-17,"10,000",11.00,1,0.0000,2,A3


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [11]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'GVREIT' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.1980,0.2015,0.2010,0.1915,0.7920,"36,000",2023-08-28,2023-09-14,,1


In [12]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 6000
WHERE name = 'GVREIT'



In [13]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [14]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'GVREIT' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.1980,0.2015,0.2010,0.1915,0.7920,"42,000",2023-08-28,2023-09-14,,1


### In case of totally new stock

In [ ]:
rcd = ['ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1]
rcd

In [ ]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [15]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks


SELECT * 
FROM stocks 
WHERE name = 'GVREIT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,108,GVREIT,0.0,0.0,I,7.0,0.0,0.0,0.0,8.7,6000,-4,4,36000,0.0,0.0,RD20pct,SET


In [16]:
new_cost = 8.5
buy_target = 6.9
sell_target = 7.4
status = 'I'
reason = 'DOS'
new_buy_qty = 6000
new_sell_qty = 6000
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(8.5, 6.9, 7.4, 'I', 'DOS', 6000, 6000)

In [17]:
#qty = 12000
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 6000, 
cost = 8.5, buy_target = 6.9, sell_target = 7.4, status = 'I', 
reason = 'DOS', qty = 6000
WHERE name = 'GVREIT'



In [18]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [19]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'GVREIT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,108,GVREIT,0.00,0.00,I,6.90,7.40,0.00,0.00,8.50,"6,000",-4,4,"42,000",0.000000,0.000000,DOS,SET


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

In [20]:
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2023-08-24' ORDER BY period, buy.name


In [21]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,MCS,2016-09-20,"78,000",15.10,6.05,"1,177,800.00","471,900.00","-705,900.00",-59.93
1,1,C1,PTTGC,2021-03-17,"6,000",64.75,36.50,"388,500.00","219,000.00","-169,500.00",-43.63
2,1,C3,SINGER,2023-01-19,"6,000",24.80,10.30,"148,800.00","61,800.00","-87,000.00",-58.47
3,1,C3,STA,2021-06-15,"5,000",40.00,13.90,"200,000.00","69,500.00","-130,500.00",-65.25
4,2,A3,AIMIRT,2023-08-17,"10,000",11.00,10.90,"110,000.00","109,000.00","-1,000.00",-0.91
5,2,B4,ASP,2022-03-07,"30,000",3.80,2.96,"114,000.00","88,800.00","-25,200.00",-22.11
6,2,A2,CPNREIT,2022-08-16,"60,000",17.55,10.80,"1,053,000.00","648,000.00","-405,000.00",-38.46
7,2,C3,DIF,2020-08-01,"30,000",14.70,9.15,"441,000.00","274,500.00","-166,500.00",-37.76
8,2,B3,GVREIT,2022-08-24,"42,000",8.50,7.35,"357,000.00","308,700.00","-48,300.00",-13.53
9,2,C3,JASIF,2018-05-17,"130,000",10.00,6.85,"1,300,000.00","890,500.00","-409,500.00",-31.50


In [22]:
file_name = 'portfolio.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

output.to_csv(output_file, header=True, index=False)
output.to_csv(data_file, header=True, index=False)
output.to_csv(one_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)